In [ ]:
import h5py
import matplotlib as mpl
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
def read_step(step: int, electronsOnly: bool = True):
    rank = 0
    df = pd.read_hdf(path + f"prt.{step:06d}_p{rank:06d}.h5", 'particles/p0/1d')
    df.drop(columns=["x", "px", "m", "w", "tag"], inplace=True)
    if electronsOnly:
        df = df[df.q == -1]
        df.drop(columns=["q"], inplace=True)
    df["step"] = step
    return df

def plot_distribution(df, x: str, y: str):
    fig, ax = plt.subplots(1, 1)
    df.plot.hexbin(x, y, gridsize=50, ax=ax);
    if x.endswith(tuple("y, z")) and y.endswith(tuple("y, z")):
        ax.set_aspect(1.)

In [ ]:
path = '/mnt/lustre/IAM851/jm1667/psc-runs/case1/single_prt/B10_n512_v+/'

## View paths of sample particles

In [ ]:
steps = range(0, 100000, 10000)
dfs = [read_step(steps[0])[::100000]]
ids = dfs[0].id
for step in steps[1:]:
    df = read_step(step)
    dfs.append(df[df.id.isin(ids)])

dfs = pd.concat(dfs)
dfs.sort_values(by=["id"]);

In [ ]:
%matplotlib widget
dfs.plot.scatter(x="y", y="z", marker='.', c="id")
# plt.quiver(dfs.y, dfs.z, dfs.py, dfs.pz)
plt.gca().set_aspect(1)

In [ ]:
step = 10000
df = read_step(step).iloc[::100]
df[df.q==-1].plot.scatter('z', 'y', marker='.')
plt.gca().set_aspect(1);
plt.tight_layout()

In [ ]:
plot_distribution(df, "py", "pz")

## View f(w, l)

In [ ]:
import bgk

In [ ]:
inputFile = bgk.readParam(path, "path_to_data", str)
B = bgk.readParam(path, "H_x", int)
maxStep = bgk.readParam(path, "nmax", int)

print(f"Path={path}")
print(f"Input{inputFile}")
print(f"max step={maxStep}")
print(f"B={B}")

In [ ]:
input = bgk.Input(inputFile)

# convert to CS units
beta = input.Te[0] ** .5
df.y = df.y / beta
df.z = df.z / beta
df.py = df.py / beta
df.pz = df.pz / beta
input.convert_to_cs_units()

In [ ]:
step = 1000000
df = read_step(step)
df

In [ ]:
df["r"] = (df.y ** 2 + df.z ** 2) ** .5
df["v_phi"] = (df.pz * df.y - df.py * df.z) / df.r
df["w"] = (df.py ** 2 + df.pz ** 2) / 2 - df.r.map(lambda r: input.interpolate_value(r, "Psi"))
df["l"] = 2 * df.r * df.v_phi - B * df.r ** 2
df

In [ ]:
%matplotlib widget
plot_distribution(df, "w", "l")